# Course Solutions

1. [Split Apply Combine Basics](#1.-Split-Apply-Combine-Basics)
1. [Split Apply Combine More](#2.-Split-Apply-Combine-More)
1. [Case Study - Counting Pandas](#Case-Study---Counting-Pandas-Solutions)

# 1. Split Apply Combine Basics

In [1]:
college = pd.read_csv('data/college.csv')

NameError: name 'pd' is not defined

### Problem 1
<span  style="color:green; font-size:16px">In the **`college`** DataFrame without using a groupby, which city name appears the most frequently?</span>

In [ ]:
# your code here
college['CITY'].value_counts().head()

### Problem 2
<span  style="color:green; font-size:16px">Does the city **`Houston`** only appear in the state of **`Texas`**?</span>

In [ ]:
# NO! There is a Houston, Missouri

# This uses slightly different boolean selection to grab a single column at the same time
college.loc[college['CITY'] == 'Houston', 'STABBR'].value_counts()

### Problem 3
<span  style="color:green; font-size:16px">Find the maximum undergraduate population for each state?</span>

In [ ]:
# your code here
college.groupby('STABBR')['UGDS'].max().head()

### Problem 4
<span  style="color:green; font-size:16px">Among colleges that have the largest undergrad population for each state, what is the difference between the most and least populous college?</span>

In [ ]:
# your code here

# from problem 3
largest_per_state = college.groupby('STABBR')['UGDS'].max()

largest_per_state.max() - largest_per_state.min()

### Problem 5: Advanced
<span  style="color:green; font-size:16px">Find the name and population of the largest college per state.</span>

In [ ]:
# couple ways to do this

#trim down the DataFrame and put the institution name in the index
college_instm = college.set_index('INSTNM')[['STABBR', 'UGDS']]
college_instm.head()

In [ ]:
# group by state and use idxmax
max_colleges = college_instm.groupby('STABBR')['UGDS'].idxmax()

In [ ]:
college_instm.loc[max_colleges]

In [ ]:
# second way

# trim data
college_trim = college[['STABBR', 'UGDS', 'INSTNM']]

# sort by state then by population descending
college_trim = college_trim.sort_values(['STABBR', 'UGDS'], ascending=[True, False])


# group by state and take the first in the group
college_trim.groupby('STABBR').first()

### Problem 6
<span  style="color:green; font-size:16px">Do distance only schools tend to have more or less student population than non-distance-only schools?</span>

In [ ]:
# They have more
college.groupby('DISTANCEONLY')['UGDS'].mean()

### Problem 7
<span  style="color:green; font-size:16px">Do distance only schools tend to be more or less religously affiliated than non-distance-only schools?</span>

In [ ]:
# Less
college.groupby('DISTANCEONLY')['RELAFFIL'].mean()

### Problem 8
<span  style="color:green; font-size:16px">What state has the lowest percentage of currently operating schools of those that have religious affiliation?</span>

In [ ]:
# your code here
cr = college[college['RELAFFIL'] == 1]

# Utah. Answer makes sense.
cr.groupby(['STABBR'])['CURROPER'].mean().sort_values().head()

### Problem 9
<span  style="color:green; font-size:16px">Trim the **`college`** DataFrame to only the 'race' columns - those beginning with **`UGDS_`**. Create a new column called **`UGDS_OTHER`** that is the sum of any race column that averages under 4% for the entire dataset.</span>

In [ ]:
# trim dataframe
df_race = college.filter(like='UGDS_')

race_average = df_race.mean()

race_average

In [ ]:
# keep only those less than 4%
other_race = race_average[race_average < .04]

other_race

In [ ]:
# get the column names
race_columns = other_race.index

race_columns

In [ ]:
# grab the columns and sum accross the rows
df_race['UGDS_OTHER'] = df_race[race_columns].sum(axis=1)

# can drop the low percentage columns
df_race.drop(race_columns, axis=1).head(10)

### Problem 10
<span  style="color:green; font-size:16px">Use the column **`UG25ABV`** and the **`quantile`** Series function to get 5 evenly spaced quantiles (use 6 numbers). Use this output to create a categorical variable using the **`cut`** function and label the bins Youngest, Young, Average, Old, Oldest and assign it to the **`AGEGROUP`** column.

Then find the average SAT math scores by AGEGROUP. Any surprising result?</span>

In [ ]:
# your code here
quants = college.UG25ABV.quantile([0, .2, .4, .6, .8, 1])
college['AGEGROUP'] = pd.cut(college.UG25ABV, quants, labels=['Youngest', 'Young', 'Average', 'Old', 'Oldest'])
college.groupby('AGEGROUP')['SATMTMID'].mean()

### Problem 11
<span  style="color:green; font-size:16px">Which are top 5 historically black colleges that have the highest white percentage?</span>

In [ ]:
# your code here
college.loc[college.HBCU == 1, ['INSTNM', 'UGDS_WHITE']].sort_values('UGDS_WHITE', ascending=False).head()

### Problem 12: Advanced
<span  style="color:green; font-size:16px">Again make a DataFrame of all the race percentage columns. Read the documentation on the **`mul`** DataFrame method and use it to multiply the race percentage DataFrame to get an actual population of each race.</span>

In [ ]:
# your code here
df_race = college.filter(like='UGDS_')

In [ ]:
df_race.mul(college['UGDS'], axis=0).round(0).head(15)

# 2. Split Apply Combine More

In [3]:
import pandas as pd
import numpy as np

pd.options.display.max_columns = 40
hou = pd.read_csv('data/coh_employee.csv')

## Problem 1
<span  style="color:green; font-size:16px">What are the 5 least common departments?</span>

In [4]:
hou.DEPARTMENT.value_counts().tail()

Houston Information Tech Svcs    9
Planning & Development           7
Mayor's Office                   5
City Controller's Office         5
Convention and Entertainment     1
Name: DEPARTMENT, dtype: int64

### Problem 2
<span  style="color:green; font-size:16px">Filter out departments with less than 50 occurences and save it to **`hou_filter`**. Then test your code by outputing the frequencies of all the remaining departments. </span>

In [5]:
hou_filter = hou.groupby('DEPARTMENT').filter(lambda df: len(df) >= 50)

hou_filter.DEPARTMENT.value_counts()

Houston Police Department-HPD     638
Houston Fire Department (HFD)     384
Public Works & Engineering-PWE    343
Health & Human Services           110
Houston Airport System (HAS)      106
Parks & Recreation                 74
Name: DEPARTMENT, dtype: int64

### Problem 3
<span  style="color:green; font-size:16px">Filter out departments from the original **`hou`** DataFrame with average salaries less than $70,000 and save it to **`hou_filter_salary`**. Then test your code by outputing the average salaries for the remaining departments.</span>

In [6]:
hou_filter_salary = hou.groupby('DEPARTMENT').filter(lambda df: df['BASE_SALARY'].mean() >= 70000)

In [7]:
# added astype(int) to remove decimals
hou_filter_salary.groupby('DEPARTMENT')['BASE_SALARY'].mean().astype(int)

DEPARTMENT
Finance                           79650
Houston Information Tech Svcs     76112
Legal Department                 104959
Mayor's Office                    86489
Name: BASE_SALARY, dtype: int32

### Problem 4
<span  style="color:green; font-size:16px">Filter *`for`* those departments from the original **`hou`** DataFrame with average salaries of at least 65,000 or having at least 25 unique position titles. Save result to **`hou_more`**</span>

In [8]:
def more(df):
    if (df['BASE_SALARY'].mean() > 65000) | (df['POSITION_TITLE'].nunique() >= 25):
        return True
    return False

hou_more = hou.groupby('DEPARTMENT').filter(more)

In [9]:
hou_more.shape

(1696, 31)

### Problem 5: Advanced
<span  style="color:green; font-size:16px">Find a way to do problem 4 without using the **`filter`** method. Make clever use of aggregate groupby and boolean logic</span>

In [ ]:
# do aggregations for each boolean piece separately
salary_grp = hou.groupby('DEPARTMENT')['BASE_SALARY'].mean()
uniq_grp = hou.groupby('DEPARTMENT')['POSITION_TITLE'].nunique()

salary_grp.head()

In [ ]:
uniq_grp.head()

In [ ]:
# create boolean criteria

deps = (salary_grp > 65000) | (uniq_grp >= 25)

deps

In [ ]:
# filter Series with itself and grab index values
deps_true = deps[deps].index.values

deps_true

In [ ]:
hou_more_check = hou[hou.DEPARTMENT.isin(deps_true)]

In [ ]:
# can check equality of dataframes with equals method
hou_more.equals(hou_more_check)

### Problem 6: Advanced
<span  style="color:green; font-size:16px">Group by department, gender and race and get the mean, min and max base salary for each group. Also get the number of unique position titles and the most frequent position title for each group. Rename each aggregation to something that makes sense. Then remove the top level of the column index. Hint: This [stackoverflow answer](http://stackoverflow.com/questions/15222754/group-by-pandas-dataframe-and-select-most-common-string-factor) will be useful </span>

In [ ]:
df = hou.groupby(['DEPARTMENT', 'GENDER','RACE']).agg({'BASE_SALARY':{'salary_mean':'mean',
                                                                'salary_min':'min',
                                                                'salary_max':'max'},
                                                 'POSITION_TITLE':{'unique_positions':'nunique',
                                                                  'most_frequent_position':lambda x: x.value_counts().index[0]}})

In [ ]:
df.columns = df.columns.droplevel(0)

In [ ]:
df.head(12)

### Problem 7
<span  style="color:green; font-size:16px"> Create a column **`is_max`** that is equal to 1 if the base salary is currently the max base salary (out of all previous rows) for that department and 0 otherwise. See sample data below.</span>

In [ ]:
hou_1 = hou[['DEPARTMENT', 'BASE_SALARY']].copy()
hou_1['is_max'] = hou_1.groupby('DEPARTMENT')['BASE_SALARY'].transform(lambda x: x == x.cummax())

In [ ]:
hou_1.head(20)

### Problem 8: Advanced
<span  style="color:green; font-size:16px"> Programatically Find the 10th occurence of 0 for **`is_max`** and return a DataFrame that ends after the tenth occurence.</span>

In [ ]:
hou_1['occur'] = hou_1.groupby('is_max').cumcount()
hou_1.head(20)

In [ ]:
idx_10 = hou_1.index[(hou_1.occur == 10) & (hou_1.is_max == 0)][0]

In [ ]:
idx_10

In [ ]:
hou_1.loc[:idx_10]

### Problem 9
<span  style="color:green; font-size:16px"> Write a function that accepts a single argument that will filter **`hou_1`** for a specific department where **`is_max`** is 1. Test your function with departments like 'Library' and 'Public Works & Engineering-PWE'.</span>

In [ ]:
def filter_dep(dep):
    criteria = (hou_1.DEPARTMENT == dep) & (hou_1.is_max == 1)
    return hou_1[criteria]

In [ ]:
filter_dep('Library')

In [ ]:
filter_dep('Public Works & Engineering-PWE')

### Problem 10
<span  style="color:green; font-size:16px">A good skill to have is to ask a difficult question for yourself and then answer it. Ask yourself a question that involes grouping and answer it.</span>

In [ ]:
# fill in your answer here

# Case Study - Counting Pandas Solutions

In [ ]:
import pandas as pd
api_tables = pd.read_html('http://pandas.pydata.org/pandas-docs/stable/api.html')

## Problem 1
<span  style="color:green; font-size:16px"> Writing a new for loop every time we want to count a new word in our dataset is cumbersome. Can you write a function that accepts the parameter **word** and returns the count of this word if it appears as in the pandas API as a functions/methods/attributes. Count a few words with it like DataFrame or MultiIndex</span>

In [ ]:
def count_functionality(word):
    return sum([table[0].str.contains(word).sum() for table in api_tables])

In [ ]:
count_functionality('Series'), count_functionality('DataFrame'), count_functionality('MultiIndex')

## Problem 2
<span  style="color:green; font-size:16px">Define a new function by modifying the above function slightly to have it return a list of all the methods</span>

In [ ]:
def list_functionality(word):
    return_list = []
    for table in api_tables:
        s = table[0] # get first column
        cur_list = s[s.str.contains(word)].tolist() # get only items with word in it and convert to list
        return_list.extend(cur_list)
    return return_list

In [ ]:
# these methods should look very familiar from the builtin python str methods
str_series = list_functionality('Series.str')
str_series

## Problem 3
<span  style="color:green; font-size:16px">Explore several of the Series `.str` methods that you should now have captured in a list on one of the API reference tables to get </span>

In [ ]:
s = api_tables[44][0]

In [ ]:
s.str.swapcase()

In [ ]:
s.str.split() # split each element by blank space

In [ ]:
# chain another str method and get the second element in the split list
s.str.split().str.get(1)

## Problem 4
<span  style="color:green; font-size:16px">Lets get some 'live' data.</span>
1. Naviate to [real clear politics](http://www.realclearpolitics.com) 
1. In the top left corner of the page, hover over the polls section and click on Clinton vs Trump
1. use pandas read_html to read in that full table at the bottom of the page and display it here in the notebook
1. use the header parameter to find the correct header instead of the default numbers
1. Inspect the info to make sure the clinton and trump data types are float64
1. add a column that calculates the difference of trump vs clinton
1. sort the dataframe by this newly created column
1. Do you see anything suspicious about the polls where Trump is leading?

In [ ]:
rcp_tables = pd.read_html('http://www.realclearpolitics.com/epolls/2016/president/us/general_election_trump_vs_clinton-5491.html', header=0)

In [ ]:
len(rcp_tables)

In [ ]:
rcp_final = rcp_tables[2]

In [ ]:
rcp_final.info()

In [ ]:
rcp_final['diff'] = rcp_final['Clinton (D)'] - rcp_final['Trump (R)']

In [ ]:
rcp_final.head(20)